# PREPARE
You don't need mongodb to run this notebook. You can just use ConversaionBufferMemmory instead. This is for testing purpose only.

To be able to run the test in this notebook, you need some file in `CompassUTD\_secret\` folder. The folder should be in the same directory as this notebook. The files are:

`google_key.json`: - Google API key for Google Cloud Platform. You can get one by making a service account
`search_secrets.json`: - Google Custom Search API key and search engine ID. You can get one by making a custom search engine with these settings:
```
{
    "GOOGLE_SEARCH_API": "REPlACE_WITH_YOUR_KEY",
    "COURSE_SEARCH_ID": "REPlACE_WITH_YOUR_KEY",
    "DEGREE_SEARCH_ID": "REPlACE_WITH_YOUR_KEY",
    "RANDOM_SEARCH_ID": "REPlACE_WITH_YOUR_KEY"
  }
```



In [ ]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()

from rich import print

import random
import string

#append the parent path to sys.path
sys.path.append(os.path.dirname(os.getcwd()))

from CompassUTD import CompassInference
from langchain.memory import ReadOnlySharedMemory, MongoDBChatMessageHistory, ConversationBufferMemory,ConversationBufferWindowMemory



agent = CompassInference()



## MongoDB Test

In [ ]:
# Use MongoDB
connection_string = f"mongodb+srv://{str(os.getenv('MONGODB_LOGIN'))}@{str(os.getenv('MONGODB_LOCATION'))}"

random_letter_string = "Test_" + "".join([random.choice(string.ascii_letters) for _ in range(11)])

print(random_letter_string)
message_history = MongoDBChatMessageHistory(
        connection_string=connection_string, session_id= random_letter_string
    )

In [ ]:

memmory_store = ConversationBufferWindowMemory(memory_key="chat_history",chat_memory=message_history)

read_only_memory = ReadOnlySharedMemory(memory=memmory_store)

# You can rerun this cell to test different messages with the same memory.
# You can reset memory by rerun the previous cell.
user_message = """What is the contact information for ECS undergraduate advising?"""

bot_message = agent.run(user_message = user_message, 
                        read_only_memory= ReadOnlySharedMemory(memory=read_only_memory)
                        )

print(bot_message)

#Save chat history

message_history.add_user_message(user_message)
message_history.add_ai_message(bot_message)


In [ ]:
assert False
# Remove all tested chat history with the sessionID of "Test_"
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
load_dotenv()

uri = f"mongodb+srv://{str(os.getenv('MONGODB_LOGIN'))}@{str(os.getenv('MONGODB_LOCATION'))}/?retryWrites=true&w=majority"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi(version="1"))
#get the database chat_history.message_store
db = client["chat_history"]
collection = db["message_store"]
#Find the session ID with the string "Test_"
# Define the string pattern to match in the key
string_pattern = "Test_"

# Create a regex query to match the key
query = { "SessionId": { "$regex": string_pattern } }
#Find all session ID with the string "Test_"
query_result = collection.delete_many(query)
